# 鹦鹉学舌

作品名称：鹦鹉学舌

作品功能：将你说的话，重复一遍，只不过声音会不同，内容也有所区别。

使用步骤：先说“虚谷号”，稍微停顿后，然后再说出你的指令。稍等片刻，虚谷号将用另一种声音，重复了你的指令。但是会出现信息传播上的损耗，有时会很搞笑。

其他说明：

    - 本案例在Snowboy语音唤醒的基础上，增加百度AI的文字识别、合成功能。无论是虚谷号还是其他迷你电脑，都需要安装百度AI库和Snowboy及其他支持库等，其中虚谷号教育版的新固件将预装Snowboy。本文档要运行在Snowboy的“examples/Python3”文件夹中。
    
    - 语音唤醒功能不需要联网，但是语音识别和语音合成功能，需要联网访问百度AI。

## 1.硬件准备

1）USB声卡+麦克风、小音箱。

2）自带声卡的摄像头（一般自带麦克风）、自带声卡的小音箱。

参考地址：

https://vvboard.readthedocs.io/zh/latest/07.plus/7.3-sound-card.html

## 1.库和参数

In [1]:
import snowboydecoder
import signal
import os
from aip import AipSpeech

interrupted = False

""" 你的 APPID AK SK """
APP_ID = "15126848"
API_KEY = "BPaS8KCk1B6Io9EqEOw1pOH3"
SECRET_KEY = "AL3B7XOmoRZojqFivCzvxuGYDDQZ7G58"

client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)

#唤醒词的模型文件名
model = 'vvboard.pmdl'

## 2.核心代码

In [2]:
# 声音到文字
def audio_to_text(wav_file):
    with open(wav_file, 'rb') as fp:
        file_context = fp.read()
    print("正在识别语音...")
    res = client.asr(file_context, 'wav', 16000, {
        'dev_pid': 1536,
    })
    if (res["err_msg"]=="success."):
        res_str = res.get("result")[0]
    else:
        res_str = "错误，没有识别出任何内容！"
    return res_str

#从文字到声音
def tts(txt):
    fname='auido.mp3'
    result = client.synthesis(txt, 'zh', 1, {'vol': 5,})
    # 识别正确返回语音二进制 错误则返回dict 参照下面错误码
    if not isinstance(result, dict):
        with open(fname, 'wb') as f:
            f.write(result)
        os.system('play '+ fname)
    os.remove(fname)

#录音结束后调用，fname为生成的录音文件
def audioRecorderCallback(fname):
    print("正在识别...")
    ret_s = audio_to_text(fname)
    #输出识别结果
    print(ret_s)
    #让虚谷号说话
    tts(ret_s)
    #删除临时录音文件
    os.remove(fname)

#唤醒时调用
def detectedCallback():
    #播放应答声
    snowboydecoder.play_audio_file(snowboydecoder.DETECT_DING)
    print('recording audio...', end='', flush=True)

def signal_handler(signal, frame):
    global interrupted
    interrupted = True

def interrupt_callback():
    global interrupted
    return interrupted

## 3.开始监听

请在安静的环境中，对虚谷号说“虚谷号”，听到“叮”的一声后，再说一句完整的话。

In [ ]:
signal.signal(signal.SIGINT, signal_handler)

#设置模型和灵敏度，sensitivity数字越大，识别越容易，但是也容易引起误判。
detector = snowboydecoder.HotwordDetector(model, sensitivity=0.38)
print('Listening... Press Ctrl+C to exit')

# main loop
detector.start(detected_callback=detectedCallback,
               audio_recorder_callback=audioRecorderCallback,
               interrupt_check=interrupt_callback,
               sleep_time=0.01)

detector.terminate()

Listening... Press Ctrl+C to exit


INFO:snowboy:Keyword 1 detected at time: 2020-02-12 17:07:21


recording audio...正在识别...
正在识别语音...


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


我叫什么名字


INFO:snowboy:Keyword 1 detected at time: 2020-02-12 17:08:23


recording audio...正在识别...
正在识别语音...
我是一块开元硬件


INFO:snowboy:Keyword 1 detected at time: 2020-02-12 17:08:49


recording audio...正在识别...
正在识别语音...
我可以学习人工智能
